# Lecture 38: Introduction to Julia

```{note}
This lecture introduces the Julia Programming Language, focusing on its core design principles and practical use in scientific computing. We will cover installation, environment setup, basic syntax, and unique features such as multiple dispatch and type stability. By the end of this lecture, you should be comfortable writing and running simple Julia programs, managing packages, and understanding why Julia is positioned as a high-performance companion to R and Python in data science and engineering workflows.
```

## Why Julia

Julia combines the readability of high‑level languages with near‑C performance via JIT compilation and multiple dispatch. It is designed for numerical and scientific computing, making it a strong companion to R and Python in this course.

<figure>
   <img src="https://julialang.org/assets/images/benchmarks.svg" alt="Julia benchmarks" style="max-width:100%;height:auto;">
   <figcaption>Julia benchmarks (source: julialang.org)</figcaption>
</figure>

### Install via `juliaup` (recommended)
`juliaup` is the official version manager (Windows/macOS/Linux). It lets you install and switch versions safely.

**Windows (PowerShell):**
```powershell
winget install julia -s msstore
juliaup add release      # installs latest stable
juliaup default release  # sets default
juliaup status           # verify channels
```

**macOS (Intel/Apple Silicon):**
```bash
brew install juliaup
juliaup add release
juliaup default release
juliaup status
```

**Linux (x86_64, aarch64):**
```bash
curl -fsSL https://install.julialang.org | sh
# Then:
juliaup add release
juliaup default release
juliaup status
```

```{tip} 
If you prefer standalone binaries, download from **julialang.org/downloads** and add `julia` to PATH. But `juliaup` is easier for managing versions.

```

### Verifying
```bash
julia --version
```
You should see something like `julia version 1.x.y`.


## Installing VS Code

### VS Code + Julia extension
1. Install **VS Code** (Windows/macOS/Linux).
2. From Extensions panel, install **Julia** (by Julia Computing).
3. Point the extension to your Julia binary if needed:
   - VS Code → Settings → search `Julia: Executable Path`
   - Example path (macOS via juliaup): `~/.juliaup/bin/julia`
4. (Optional) Install **CodeLLDB** for debugging.

### Quality-of-life extensions
- **Inline Inlay Hints** and **Error Lens** (already bundled in Julia ext.)
- **Jupyter** if you prefer notebooks (`.ipynb`) in VS Code

### REPL workflow
- Open a `.jl` file → `Alt+J` then `Alt+O` (or `Command+J` on macOS) to open REPL
- Send line/selection with `Shift+Enter`



## Hello World!


In [2]:
# Hello World in Julia
println("Hello, CE5540!")

Hello, CE5540!


## Data Types in Julia

Common scalar types:
- `Int64`, `Float64`, `Bool`
- `Char`, `String`
- `Missing`, `Nothing`
- `ComplexF64`, `Rational{Int}`

Use `typeof(x)` to inspect types.


In [3]:
# Character & String
α = 'J'                 # Char
β = "CE5540"            # String
println("α = ", α, " :: ", typeof(α))
println("β = ", β, " :: ", typeof(β))

# Integers & Floats
nᵢ = 42                  # Int64 (on 64-bit platforms)
nᶠ = 3.14                # Float64
println("nᵢ :: ", typeof(nᵢ), ", nᶠ :: ", typeof(nᶠ))

# Boolean
φ = nᵢ ≥ 40
println("φ = ", φ, " :: ", typeof(φ))

# Complex & Rational
nᶜ = 2 + 3im
nʳ = 3 // 7
println("nᶜ = ", nᶜ, " :: ", typeof(nᶜ))
println("nʳ = ", nʳ, " :: ", typeof(nʳ))

# Missing / Nothing
m = missing
n = nothing
println("m :: ", typeof(m), ", n :: ", typeof(n))

α = J :: Char
β = CE5540 :: String
nᵢ :: Int64, nᶠ :: Float64
φ = true :: Bool
nᶜ = 2 + 3im :: Complex{Int64}
nʳ = 3//7 :: Rational{Int64}
m :: Missing, n :: Nothing


## Data Structures in Julia

- **Tuple** (immutable), **NamedTuple**
- **Vector/Matrix** (1D/2D `Array`), **range** objects
- **Dict** (hash map), **Set**
- **DataFrame** (from `DataFrames.jl`)

Note: Many operations are not automatically vectorized; prefer **broadcasting** with the dot `.` operator (e.g., `sin.(x)`).


In [4]:
# Tuples and NamedTuples
t₁ = (1, 2.0, "a")
t₂ = (a = 1, b = 2.0, c = "a")
println(t₁, " :: ", typeof(t₁))
println(t₂, " :: ", typeof(t₂))

# Arrays (Vector/Matrix) and ranges
V = [1, 2, 3, 4]             # Vector{Int}
M = [1 2 3; 4 5 6]           # 2×3 Matrix{Int}
R = 0:2:10                   # range 0,2,4,6,8,10
println(V, " :: ", typeof(V))
println(M, " :: ", typeof(M))
println(collect(R), " :: ", typeof(R))

# Dict and Set
D = Dict("a" => 1, "b" => 2)
S = Set([1,2,2,3,3,3])
println(D, " :: ", typeof(D))
println(S, " :: ", typeof(S))

# Broadcasting with dot notation
X = [1.0, 4.0, 9.0]
Y = sqrt.(X)    # element-wise sqrt
println(Y)

(1, 2.0, "a") :: Tuple{Int64, Float64, String}
(a = 1, b = 2.0, c = "a") :: @NamedTuple{a::Int64, b::Float64, c::String}
[1, 2, 3, 4] :: Vector{Int64}
[1 2 3; 4 5 6] :: Matrix{Int64}
[0, 2, 4, 6, 8, 10] :: StepRange{Int64, Int64}
Dict("b" => 2, "a" => 1) :: Dict{String, Int64}
Set([2, 3, 1]) :: Set{Int64}
[1.0, 2.0, 3.0]


## Control Flow

Julia supports standard `if/elseif/else`, `for`, and `while` constructs.


In [5]:
# If / elseif / else
x = 10
if x > 10
    println("x > 10")
elseif x == 10
    println("x == 10")
else
    println("x < 10")
end

# For loop
s = 0
for k ∈ 1:5
    s += k
end
println("Sum 1..5 = ", s)

# While loop
i = 1
p = 1
while i ≤ 5
    p *= i
    i += 1
end
println("Product 1..5 = ", p)

x == 10
Sum 1..5 = 15
Product 1..5 = 120


## Writing Functions in Julia (and Multiple Dispatch)

Functions can be type-annotated.

In [6]:
# Factorial (iterative)
function fᵢ(n::Integer)
    n < 0 && error("n must be non-negative")
    r = one(n)
    for k ∈ 2:n
        r *= k
    end
    return r
end

# Factorial (recursive)
function fᵣ(n::Integer)
    n < 0 && error("n must be non-negative")
    return n ≤ 1 ? one(n) : n * fᵣ(n-1)
end

println("5! iterative = ", fᵢ(5))
println("5! recursive = ", fᵣ(5))

5! iterative = 120
5! recursive = 120


## Multiple Dispatch

Julia selects methods at runtime based on the types of arguments.

In [9]:
# Multiple dispatch example
f(x::Int, y::Int) = x + y
f(x::Float64, y::Float64) = x + y
f(x::String, y::String) = string(x, y)

println("f(2, 3) -> ", f(2,3), " :: ", typeof(f(2,3)))
println("f(2.0, 3.0) -> ", f(2.0,3.0), " :: ", typeof(f(2.0,3.0)))
println("f(`a`, 3) -> ", f("a", 3), " :: ", typeof(f("a",3)))

# Method lookup with @which (reveals the exact method that will be called)
# Try with our multiple-dispatch function f:
println(@which f(1, 2))          # both Int → f(x::Int, y::Int)
println(@which f(1.0, 2.0))      # both Float64 → f(x::Float64, y::Float64)
println(@which f("1", "2.0"))        # promotes to Float64 → f(x::Float64, y::Float64)

# Works for Base methods too
println(@which +(1, 2))          # shows which method of + is used
println(@which *(1.0, 2))        # multiplication method for Float64 × Int

f(2, 3) -> 5 :: Int64
f(2.0, 3.0) -> 5.0 :: Float64
f(`a`, 3) -> a3 :: String
f(x::Int64, y::Int64) @ Main c:\Users\Anmol Pahwa\OneDrive - IIT-Madras(IC&SR)\Academia\IIT Madras\Teaching\CE5540\book\content\jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_X24sZmlsZQ==.jl:2
f(x::Float64, y::Float64) @ Main c:\Users\Anmol Pahwa\OneDrive - IIT-Madras(IC&SR)\Academia\IIT Madras\Teaching\CE5540\book\content\jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_X24sZmlsZQ==.jl:3
f(x::String, y::String) @ Main c:\Users\Anmol Pahwa\OneDrive - IIT-Madras(IC&SR)\Academia\IIT Madras\Teaching\CE5540\book\content\jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_X24sZmlsZQ==.jl:4
+(x::T, y::T) where T<:Union{Int128, Int16, Int32, Int64, Int8, UInt128, UInt16, UInt32, UInt64, UInt8} @ Base int.jl:87
*(x::Number, y::Number) @ Base promotion.jl:430


## Type Stability

In [8]:
# Type stability matters for performance. A function is *type-stable*
# if the compiler can infer a concrete return type from the input types.

# Type-stable example
f₁(v::Vector{Int})::Int = sum(v)

# Type-unstable example: operates on abstractly-typed inputs
f₂(v::Vector{Any}) = sum(v)

# Construct two vectors
v₁ = [1, 2, 3, 4, 5]                # Vector{Int}
v₂ = Any[1, 2.0, 3, 4.0, 5]         # Vector{Any} (mixed types)

# Inspect type inference
@code_warntype f₁(v₁)       # should show a concrete return type (Int64 on 64-bit)
@code_warntype f₂(v₂)       # likely to show `Any`-typed intermediate/return

# Tips for stability:
# - Use concrete container element types (Vector{Float64}, Vector{Int}, etc.)
# - Avoid changing the type of a variable inside a function
# - Annotate return types where it clarifies intent (not always necessary)

MethodInstance for f₁(::Vector{Int64})
  from f₁(v::Vector{Int64}) @ Main c:\Users\Anmol Pahwa\OneDrive - IIT-Madras(IC&SR)\Academia\IIT Madras\Teaching\CE5540\book\content\jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_X16sZmlsZQ==.jl:5
Arguments
  #self#::Core.Const(Main.f₁)
  v::Vector{Int64}
Locals
  @_3::Int64
Body::Int64
1 ─ %1  = Main.Int::Core.Const(Int64)
│   %2  = Main.sum::Core.Const(sum)
│   %3  = (%2)(v)::Int64
│         (@_3 = %3)
│   %5  = @_3::Int64
│   %6  = (%5 isa %1)::Core.Const(true)
└──       goto #3 if not %6
2 ─       goto #4
3 ─       Core.Const(:(@_3))
│         Core.Const(:(Base.convert(%1, %9)))
└──       Core.Const(:(@_3 = Core.typeassert(%10, %1)))
4 ┄ %12 = @_3::Int64
└──       return %12

MethodInstance for f₂(::Vector{Any})
  from f₂(v::Vector{Any}) @ Main c:\Users\Anmol Pahwa\OneDrive - IIT-Madras(IC&SR)\Academia\IIT Madras\Teaching\CE5540\book\content\jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_X16sZmlsZQ==.jl:8
Arguments
  #self#::Core.Const

## Writing Fast Julia: Type Stability & Broadcasting

- Type stability: Ensure functions return a consistent type. Use `@code_warntype` in the REPL to diagnose instabilities.
- Avoid global state: Wrap code in functions; globals are slow.
- Use broadcasting: Prefer `f.(x)` over manual loops when applying scalar functions element-wise.
- Preallocate: For large loops, preallocate arrays to avoid repeated memory allocations.

## Creating a Julia Project

In [ ]:
# Julia environments and package management with Pkg
# Run in the Julia REPL or VS Code Julia REPL

import Pkg

# 1) Create/activate a new project (creates Project.toml and Manifest.toml)
Pkg.activate("ABM101")  # local env in ./ABM101
# Alternatively: Pkg.generate("ABM101")  # scaffolds a new package project

# 2) Add packages
Pkg.add([
    "DataFrames",
    "CSV",
    "Plots",
    "Distributions",
    "StatsBase"
])

# 3) Check status / resolve
Pkg.status()
Pkg.resolve()

# 4) Pin (optional), update, and instantiate (recreate exact environment on new machine)
# Pkg.pin("DataFrames")
Pkg.update()
Pkg.instantiate()

# 5) Using packages in code (after activation)
using DataFrames, CSV, Plots, Distributions, StatsBase

# 6) Best practice for reproducibility:
#    - Commit Project.toml & Manifest.toml to version control
#    - Colleagues call `Pkg.activate("."); Pkg.instantiate()` to match your environment
